In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import requests
import pandas as pd
import json
import time
import random
import re
import sqlite3

In [ ]:
driverDir = r'/media/datngo/Driver1/Ubuntu setup/chromedriver_linux64/chromedriver'

## Scrape loans' data and manipulate data and create table kivaLoans 

In [ ]:
sectorsCode = range(1,18)
genders = ['male','female']
startPageLink = 'https://www.kiva.org/lend?gender={0}&sector={1}'
loanData = []
driver = webdriver.Chrome(executable_path = driverDir)
for i, sector in enumerate(sectorsCode):
    for j, gender in enumerate(genders):
        pageNo = 1
        driver.get(startPageLink.format(gender,sector))
        time.sleep(10)
        viewList = driver.find_element_by_xpath("//label[@for = 'view-buttons-list']")
        viewList.click()
        time.sleep(10)
        while True:
            if len(driver.find_elements_by_xpath("//div[@class = 'loan-card-2 row']")) > 0: # check whether a page contains a loan information
                for loanElement in driver.find_elements_by_xpath("//div[@class = 'loan-card-2 row']"): # get all div of loan in a page
                    borrowLink = loanElement.find_element_by_xpath(".//a[@class = 'loan-card-2-borrower-name']").get_attribute('href')
                    loanText = loanElement.text.split('\n')
                    loanData.append(tuple([i] + [j] + [borrowLink]  + loanText[:-1]))
            else:
                break
            try:
                nextPage = driver.find_element_by_xpath("//a[@class = 'next button secondary ']")
                nextPage.click()
            except Exception as e2:
                print(gender, sector, pageNo,len(loanData))
                print("End of sector")
                break
            print(gender, sector, pageNo,len(loanData))
            pageNo += 1
            time.sleep(10)

In [ ]:
# manipulate crawled data to useful format
kiva_df = pd.DataFrame(loanData, columns = ['sectorId', 'genderId', 'loanUrl', 'borrowerName', 'country',
       'loanDescription', 'lendingPartner', 'loanLength', 'amountToGo'])
kiva_df['loanId'] = kiva_df['loanUrl'].apply(lambda x: int(x.split('/')[-1]))
kiva_df['loanAmount_USD'] = kiva_df['loanDescription'].apply(lambda x: int(x.split('$')[1].split()[0].replace(',','')))
kiva_df['loanDescriptionText'] = kiva_df['loanDescription'].apply(lambda x: x[re.search('^A loan of \$[0-9\,]*',x).end():]) 
kiva_df['loanLengthNumber'] = kiva_df['loanLength'].apply(lambda x: int(x[:re.search(' months',x).start()]))

In [ ]:
# load kiva loan data to database
conn = sqlite3.connect('kivaloans.db')
kiva_df[['sectorId','genderId','loanUrl','country','loanDescriptionText','lendingPartner'
        ,'loanLengthNumber','amountToGo','loanId','loanAmount_USD']].to_sql('kivaLoans', conn, if_exists='replace', index = True)
conn.commit()
conn.close()

## Scrape countries' data and create table countries

In [ ]:
# Crawling country list
driver = webdriver.Chrome(executable_path = driverDir)
driver.get('https://www.kiva.org/lend?kexpn=lend_filter.lend_filter_versions&kexpv=c')
time.sleep(5)
driver.find_element_by_xpath("//a[@class = 'filter-menu-button button setting left-off-canvas-toggle']").click()
time.sleep(5)
driver.find_element_by_xpath("//a[@data-reveal-id = 'countrySelectModal']").click()
time.sleep(5)
countryList = []
for regionElement in driver.find_elements_by_xpath("//section[@class = 'region-name']"):
    for countryElement in regionElement.find_elements_by_xpath(".//div[@class = 'country filter checkbox-input']"):
        countryCode = countryElement.find_element_by_xpath(".//input[@class = 'countryCheckbox']").get_attribute('value')
        countryName = countryElement.text[:re.search(' \([0-9]*\)',countryElement.text).start()]
        countryList.append((countryName,countryCode,regionElement.text.split('\n')[0][:-11]))

In [ ]:
# load country info into db
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
# c.execute("DROP TABLE countries")
c.execute("""CREATE TABLE countries(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            countryName TEXT,
            countryCode TEXT,
            regionName TEXT
            )""")
conn.commit()
c.executemany("INSERT INTO countries(countryName,countryCode, regionName) VALUES(?,?,?)",countryList)
conn.commit()
conn.close()

## Scrape sectors' data and create table sectors

In [ ]:
# load sector table into db
sectorCodes = [1,9,5,14,15,17,12,6,10,8,16,7,4,3,13]
sectorNames = ['Agriculture','Arts','Clothing','Construction','Education','Entertainment','Food','Health','Housing',
              'Manufacturing','Personal use','Retail','Services','Transportation','Wholesale']
sectorIds = list(map(lambda x: x - 1, sectorCodes))
sectorRecords = []
for sectorId,code,name in zip(sectorIds,sectorCode,sectorName):
    sectorRecords.append((sectorId,code,name))

conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
# c.execute("DROP TABLE sectors")
c.execute("""CREATE TABLE sectors(
            sectorId INTEGER PRIMARY KEY,
            sectorCode TEXT,
            sectorName TEXT)""")
conn.commit()
c.executemany("INSERT INTO sectors(sectorId, sectorCode, sectorName) VALUES(?,?,?)",sectorRecords)
conn.commit()
conn.close()

## Create table genders

In [ ]:
# load gender table into db
genderRecords = [(0, 'male'),(1, 'female')]
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("""CREATE TABLE genders(
            genderId INTEGER PRIMARY KEY,
            gender TEXT)""")
c.executemany("INSERT INTO genders(genderId, gender) VALUES (?,?)",genderRecords)
conn.commit()
conn.close()

## Create table kivaLoansUpdateTemp for updating new loans

In [ ]:
# create the temporary update table
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("""
            CREATE TABLE kivaLoansUpdateTemp (
            loanId INTEGER,
            borrowerName TEXT,
            loanLengthNumber INTEGER,
            borrowerGender TEXT,
            amountRaised INTEGER,
            sector TEXT,
            loanDescription TEXT,
            country TEXT,
            loanAmount INTEGER,
            lendingPartner TEXT,
            loanUrl TEXT,
            genderId INTEGER,
            sectorId INTEGER,
            countryId INTEGER,
            loanAmountToGo INTEGER,
            createDate TEXT
            )""")
conn.commit()
conn.close()

## Create table users

In [ ]:
# create table users for userName and password
import sqlite3
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("""
            CREATE TABLE users (
            userId INTEGER PRIMARY KEY AUTOINCREMENT, 
            userName TEXT,
            password TEXT
            )""")
conn.commit()
conn.close()

## Create table countryLatLon to add iso3 code for map chart

In [ ]:
# create table country Lat Long to load country lattitude and longtitude
conn = sqlite3.connect('kivaloans.db')
countryLatLon_df = pd.read_csv("worldcountries.csv")
conn = sqlite3.connect('kivaloans.db')
countryLatLon_df.to_sql('countryLatLon',conn, if_exists='replace', index = True)
conn.commit()
conn.close()

In [ ]:
# add iso3 code into table countries
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("""ALTER TABLE countries ADD COLUMN iso3 TEXT""")
c.execute("""UPDATE countries 
            SET iso3 = (SELECT iso3 FROM countryLatLon WHERE LOWER(iso2) = countryCode)""")
conn.commit()

## Add, update columns and create foreign keys

In [ ]:
# Add and update columns countryId and createdDate in table kivaLoans
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("ALTER TABLE kivaLoans ADD COLUMN countryId TEXT")
c.execute ("UPDATE kivaLoans SET countryId = (SELECT id FROM countries WHERE countryName = country)")
c.execute("ALTER TABLE kivaLoans ADD COLUMN createdDate TEXT")
c.execute("UPDATE kivaLoans SET createdDate = '2021-01-20'")
conn.commit()

In [ ]:
# Add foreign keys in kivaLoans table
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute ("""ALTER TABLE kivaLoans RENAME TO kivaLoans_old""");
c.execute("PRAGMA foreign_keys = ON")
c.execute("DROP TABLE kivaLoans")
c.execute ("""
            CREATE TABLE kivaLoans(
            id INTEGER PRIMARY KEY,
            sectorId INTEGER,
            genderId INTEGER,
            loanUrl TEXT,
            country TEXT,
            loanDescriptionText TEXT,
            lendingPartner TEXT,
            loanLengthNumber TEXT,
            amountToGo TEXT,
            loanId INTEGER,
            loanAmount_USD INTEGER,
            countryId INTEGER,
            createdDate TEXT,
            FOREIGN KEY (genderId) REFERENCES genders(genderId),
            FOREIGN KEY (countryId) REFERENCES countries(id),
            FOREIGN KEY (sectorId) REFERENCES sectors(sectorId))""")
c.execute ("INSERT INTO kivaLoans SELECT * FROM kivaLoans_old")
c.execute("DROP TABLE kivaLoans_old")
conn.commit()

## Create indexes

In [ ]:
create indexes
genderRecords = [(0, 'male'),(1, 'female')]
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("""CREATE INDEX sectorId ON kivaLoans(sectorId)""")
c.execute("""CREATE INDEX countryId ON kivaLoans(countryId)""")
c.execute("""CREATE INDEX genderId ON kivaLoans(genderId)""")
c.execute("""CREATE INDEX kivaLoanId ON kivaLoans(loanId)""")
c.execute("""CREATE INDEX genderId_gender ON genders(genderId)""")
c.execute("""CREATE INDEX countryId_country ON countries(id)""")
c.execute("""CREATE INDEX sectorId_sector ON sectors(sectorId)""")
conn.commit()
conn.close()

## Create a view

In [ ]:
# Create a view of loan info by joining tables
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
# c.execute ("DROP VIEW v_loanInfoSum")
c.execute("""
            CREATE VIEW v_loanInfoSum
            AS SELECT 
            L.id,L.loanUrl,L.country,L.loanDescriptionText,L.lendingPartner
            ,L.loanLengthNumber, L.loanAmount_USD,L.loanId, L.amountToGoNumber
            ,C.countryCode, C.regionName
            ,G.gender, S.sectorCode, S.sectorName
            FROM kivaLoans L 
            LEFT JOIN countries C ON L.countryId = C.id
            LEFT JOIN sectors S ON L.sectorId = S.sectorId
            LEFT JOIN genders G ON L.genderId = G.genderId""")
conn.commit()
conn.close()

## Manipulate data in table KivaLoans using custom functions

In [ ]:
conn = sqlite3.connect('kivaloans.db')
c = conn.cursor()
c.execute("ALTER TABLE kivaLoans ADD COLUMN amountToGoNumber INTEGER")
def amountExtract(amt):
    try:
        return int(re.search('\$[0-9\,]*',amt).group(0).replace(',','').replace('$',''))
    except: 
        return None
conn.create_function("amountExtract",1,amountExtract)
c.execute("UPDATE kivaLoans SET amountToGoNumber = amountExtract(amountToGo)")
conn.commit()

## Remove duplicate data

In [ ]:
# remove duplicate rows based on loan Id
c.execute("""DELETE 
            FROM kivaLoans
            WHERE id IN 
            (SELECT id FROM 
            (SELECT id,loanId, RANK () OVER (PARTITION BY loanId ORDER BY id ASC) as rk
            FROM kivaLoans) AS t
            WHERE t.rk > 1)
""")
conn.commit()